In [2]:
from tqdm.auto import tqdm
import pandas as pd
tqdm.pandas()

In [3]:
df = pd.read_csv("../data/train.csv")
df

,comment_id,comment_text,Sub1_Toxic,Sub2_Engaging,Sub3_FactClaiming
0,1,Ziemlich traurig diese Kommentare zu lesen. Ih...,0,0,0
1,2,"Sag ich doch, wir befeuern den Klimawandel. Ra...",0,1,1
2,3,"Schublade auf, Schublade zu. Zu mehr Denkleist...",1,0,0
3,4,Dummerweise haben wir in der EU und in der USA...,0,0,1
4,5,"""So lange Gewinnmaximierung Vorrang hat, wird ...",0,0,0
...,...,...,...,...,...
3239,3240,Hier mal eine Info. Flüchtlinge werden 10 km v...,0,0,0
3240,3241,@USER.aha .Mal abwarten kommt bei uns auch .Fi...,1,0,1
3241,3242,@USER .So ist es,0,0,0
3242,3243,@USER .Die warten da,0,0,0


In [4]:
with open("../data/emojis_dict.json") as f:
  import json
  emoji_dict=json.load(f)

In [5]:
import re
from nltk.corpus import stopwords
df['user']=df['comment_text'].apply(lambda comment: 1 if "@USER" in comment else 0)
df['mention']=df['comment_text'].apply(lambda comment: 1 if "@MEDIUM" in comment else 0)
df['mod']=df['comment_text'].apply(lambda comment: 1 if "@MODERATOR" in comment else 0)
df['total_length'] = df['comment_text'].apply(len)
df['num_words'] = df.comment_text.str.count('\S+')
df['all_caps']= df['comment_text'].apply(lambda comment: sum(1 for c in comment.split() if c.isupper() and c not in ["@USER", "@MEDIUM", "@MODERATOR"]))/df['num_words']
df['emoji']=df['comment_text'].apply(lambda comment: len(re.findall(u'[\U0001f600-\U0001f650]', comment)))/df['num_words']
df['caps']= df['all_caps'].apply(lambda num: 1 if num > 0 else 0 )
df['capitals'] = df['comment_text'].apply(lambda comment: sum(1 for c in comment if c.isupper()))
df['caps_vs_length'] = df.apply(lambda row: float(row['capitals'])/float(row['total_length']),
                                axis=1)
df['num_unique_words'] = df['comment_text'].apply(lambda comment: len(set(w for w in comment.split())))
df['words_vs_unique'] = df['num_unique_words'] / df['num_words']
df['num_urls']=df['comment_text'].apply(lambda comment: len(re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', comment)))
df['!']=df['comment_text'].apply(lambda comment : comment.count("!"))/df['num_words']
df['?']=df['comment_text'].apply(lambda comment : comment.count('?'))/df['num_words']
import string
c=0
def transcribe(text):
  global c
  new_text=text
  for emoji,de_text in emoji_dict.items():
    new_text=new_text.replace(emoji," "+de_text+" ")
    if new_text!=text:
      c+=1
  return new_text
df['punc'] = df["comment_text"].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))/df['num_words']
df["text"]= df['comment_text'].progress_apply(lambda x: transcribe(x))
print(c)
df["text"]= df['text'].map(lambda x: x.replace("@USER",""))
df["text"]= df['text'].map(lambda x: x.replace("@MEDIUM",""))
df["text"]= df['text'].map(lambda x: x.replace("@MODERATOR",""))
df['text'] = df['text'].map(lambda x: re.sub('\\n',' ',str(x)))
# remove IP addresses or user IDs
df['text'] = df['text'].map(lambda x: re.sub("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}",'',str(x)))
# lower uppercase letters
df['text'] = df['text'].map(lambda x: str(x).lower())
#remove http links in the text
df['text'] = df['text'].map(lambda x: re.sub("(http://.*?\s)|(http://.*)",'',str(x)))
#remove all punctuation except for apostrophe (')
df['text'] = df['text'].map(lambda x: re.sub('[!"#$%&\()*+,-/:;<=>?@[\\]^_`{|}~]','',str(x)))
df['stop']= df["text"].progress_apply(lambda comment: sum(1 for w in comment.split() if w in stopwords.words('german')))/df['num_words']

  0%|          | 0/3244 [00:00<?, ?it/s]

1238560


  0%|          | 0/3244 [00:00<?, ?it/s]

In [6]:
df

,comment_id,comment_text,Sub1_Toxic,Sub2_Engaging,Sub3_FactClaiming,user,mention,mod,total_length,num_words,...,capitals,caps_vs_length,num_unique_words,words_vs_unique,num_urls,!,?,punc,text,stop
0,1,Ziemlich traurig diese Kommentare zu lesen. Ih...,0,0,0,0,0,0,322,51,...,13,0.040373,48,0.941176,0,0.0,0.019608,0.117647,ziemlich traurig diese kommentare zu lesen ihr...,0.509804
1,2,"Sag ich doch, wir befeuern den Klimawandel. Ra...",0,1,1,0,0,0,148,23,...,6,0.040541,22,0.956522,0,0.0,0.000000,0.173913,sag ich doch wir befeuern den klimawandel rauc...,0.565217
2,3,"Schublade auf, Schublade zu. Zu mehr Denkleist...",1,0,0,0,0,0,79,13,...,4,0.050633,12,0.923077,0,0.0,0.000000,0.230769,schublade auf schublade zu zu mehr denkleistun...,0.538462
3,4,Dummerweise haben wir in der EU und in der USA...,0,0,1,0,0,0,115,21,...,12,0.104348,19,0.904762,0,0.0,0.000000,0.142857,dummerweise haben wir in der eu und in der usa...,0.571429
4,5,"""So lange Gewinnmaximierung Vorrang hat, wird ...",0,0,0,0,0,0,100,16,...,4,0.040000,16,1.000000,0,0.0,0.000000,0.250000,so lange gewinnmaximierung vorrang hat wird si...,0.562500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3239,3240,Hier mal eine Info. Flüchtlinge werden 10 km v...,0,0,0,0,0,0,130,22,...,8,0.061538,22,1.000000,0,0.0,0.000000,0.090909,hier mal eine info flüchtlinge werden 10 km vo...,0.454545
3240,3241,@USER.aha .Mal abwarten kommt bei uns auch .Fi...,1,0,1,1,0,0,221,34,...,13,0.058824,32,0.941176,0,0.0,0.000000,0.205882,aha mal abwarten kommt bei uns auch firmen ent...,0.500000
3241,3242,@USER .So ist es,0,0,0,1,0,0,16,4,...,5,0.312500,4,1.000000,0,0.0,0.000000,0.500000,so ist es,0.750000
3242,3243,@USER .Die warten da,0,0,0,1,0,0,20,4,...,5,0.250000,4,1.000000,0,0.0,0.000000,0.500000,die warten da,0.500000


In [7]:
import json
f1 = open("../data/positives.json")
f2 = open("../data/negatives.json")
positive = json.load(f1)
negative = json.load(f2)
positive_words = [i[0] for i in positive]
positive_scores = [i[1] for i in positive]
negative_words = [i[0] for i in negative]
negative_scores = [i[1] for i in negative]

pos_score = []
neg_score = []

def find_negative(text):
  words = text.split()
  count = 0
  score = 0
  for w in words:
    try:
      indx = negative_words.index(w.lower())
      count+=1
      score+= negative_scores[indx]
    except:
      pass
  neg_score.append(abs(score))
  return count

def find_positive(text):
  words = text.split()
  count = 0
  score = 0
  for w in words:
    try:
      indx = positive_words.index(w.lower())
      count+=1
      score+= positive_scores[indx]
    except:
      pass
  pos_score.append(abs(score))
  return count

df["num_negative_words"] = df["text"].progress_apply(lambda x: find_negative(x))
df["negativity_score"] = neg_score
df["num_positive_words"] = df["text"].progress_apply(lambda x: find_positive(x))
df["positivity_score"] = pos_score
df['neg']=df["negativity_score"]/df["num_negative_words"]
df['pos']=df["positivity_score"]/df["num_positive_words"]
df['pos']=df['pos'].fillna(0)
df['neg']=df['neg'].fillna(0)

  0%|          | 0/3244 [00:00<?, ?it/s]

  0%|          | 0/3244 [00:00<?, ?it/s]

In [8]:
import language_tool_python
tool = language_tool_python.LanguageToolPublicAPI('de')
match_u=[]
def get_errors(text):
  global match_u
  matches = tool.check(text)
  matches=[match for match in matches if match.ruleId not in ['DE_CASE','DE_DASH','DE_DU_UPPER_LOWER','DE_PHRASE_REPETITION','DE_SENTENCE_WHITESPACE','EMAIL','GERMAN_WORD_REPEAT_BEGINNING_RULE',
 'GERMAN_WORD_REPEAT_RULE','PLURAL_APOSTROPH','PUNCTUATION_PARAGRAPH_END','UNPAIRED_BRACKETS',
 'UPPERCASE_SENTENCE_START']]
  match_ui=[match.ruleId for match in matches]
  match_u.extend(match_ui)
  return len(matches)/len(text)
df['error']=df['comment_text'].progress_apply(lambda comment: get_errors(comment))

self._url: https://api.languagetool.org/v2/


  0%|          | 0/3244 [00:00<?, ?it/s]

In [9]:
german = {'present':['nutze ab',
                      'nutzt ab',
                      'nutzen ab',
                      'gehe aus',
                      'gehst aus',
                      'geht aus',
                      'gehen aus',
                      'rufe an',
                      'rufst an',
                      'ruft an',
                      'rufen an',
                      'rufe an',
                      'rufst an',
                      'ruft an',
                      'rufen an',
                      'gehe auf',
                      'gehst auf',
                      'geht auf',
                      'gehen auf',
                      'breche zusammen',
                      'brichst zusammen',
                      'bricht zusammen',
                      'brechen zusammen',
                      'brecht zusammen',
                      'arbeite',
                      'arbeitest',
                      'arbeitet',
                      'arbeiten',
                      'bin',
                      'bist', 
                      'ist', 
                      'sind',#typo
                      'seid',#typo
                      'sein',
                      'komme',
                      'kommst',
                      'kommt',
                      'kommen',
                      'steige',
                      'steigst',
                      'steigt',
                      'steigen',
                      'bekomme',
                      'bekommst',
                      'bekommt',
                      'bekommen',
                      'mache',
                      'machst',
                      'macht',
                      'machen',
                      'nutzen',
                      'nutze',
                      'nutzt',
                      'habe',
                      'hast',
                      'hat',
                      'haben',
                      'habt',
                      'fühle',
                      'fühlst',
                      'fühlt',
                      'fühlen',
                      'fuehlst',
                      'gewinne',
                      'gewinnst',
                      'gewinnt',
                      'gewinnen',
                      'regne',
                      'regnest',
                      'regnet',
                      'regnen',
                      'kaufe',
                      'kaufst',
                      'kauft',
                      'kaufen',
                      'sterbe',
                      'stirbst',
                      'stirbt',
                      'sterben',
                      'sterbt',
                      'reise',
                      'reist',
                      'reisen',
                      "rijs",#typo
                      'ruft',
                      'rufst',
                      'rufe',
                      'rufen',
                      'gehe',
                      'gehst',
                      'geht',
                      'gehen',
                      'liebe',
                      'liebst',
                      'liebt',
                      'lieben',
                      'lebe', 
                      'lebst', 
                      'lebt', 
                      'leben'
            ],
    'future': 
            ['werde',
              'wirst',
              'wird',
              'werden',
              'werdet',
              'werden',
              'wirdt',#typo
              'werde',
              'wirst',
              'wird',
              'werden',
              'werdet',
              'werden',
              'will',
              'willst',
              'wollen',
              'wollt',
              'wollen',
              ##konjunktiv ii of mogen meaning to want/would like
              'möchte',
              'möchtest',
              'möchten',
              'möchtet',
              'hoffe',#hope
              'hoffst',
              'hofft',
              'hoffen',
              'plane',
              'planst',
              'plant',
              'plannen',
              ],
    'uncertainity':
                 [#indicative of können
                  'kann',
                  'kannst',
                  'könnt',
                  'können',
                  
                  #konjunktiv ii of können
                  'könnte',
                  'könntest',
                  'könnten',
                  'könntet',
                  
                  #may have some epistemic use, but not typical, as in english ''should' (Nuyts 2000). Both konjunctiv ii and indicative included    
                  'sollen',
                  'soll',
                  'sollst',
                  'sollt',
                  
                  #konjunktiv ii
                  'sollte',
                  'solltest',
                  'sollten',
                  'solltet',
                  
                  #indicative dürfen cannot have epistemic uses, only deontic (Nuyts, 2000)
                  #'darf',
                  #'darfst',
                  #'dürfen',
                  #'dürft',
                  
                  #konjunktiv ii of ¨dürfen
                  'dürfte',
                  'dürftest',
                  'dürften',
                  'dürftet',
                  
                  #mogen 'may' in the indicative has epistemic uses, but not konjunktiv ii (Nuyts 2000)
                  'mag',
                  'magst',
                  'mögen',
                  'mögt',
                  
                  
                  ##konjuntiv of werden, i.e. 'would' with epistemic uses: according to informant/coder
                  'würde',
                  'würdest',
                  'würden',
                  'würdet',
                  
                  #konjunktiv ii of müssen has epistemic uses, like 'should' (Mortelmans 2000).
                  'müßte',
                  'müßtest',
                  'müßten',
                  'müßtet',
                  'müsste',
                  'müsstest',
                  'müssten',
                  'müsstet',
                  'unter umständen',
                  'annehmbar',#presumably (Nuyts 2000)
                  'eventuell',
                  'anscheinend',
                  'gegebenenfalls',
                  'wahrscheinlichkeit',#probability -- informant coder
                  'möglich',
                  'möglicherweise',
                  'offenbar',
                  'scheinbar',#seemingly
                  #'vielleicht',
                  'vermutlich',#presumably (Nuyts 2000)
                  'wahrscheinlich',
                  'womöglich',
                  'wohl',
                  'vielleicht',
                  'aber',
                  'auch',
                  'bloß',
                  'denn',
                  'doch',
                  'eigentlich',
                  'eben',
                  'etwa',
                  'einfach',
                  'erst',
                  'halt',
                  'ja',
                  'nun',
                  'mal',
                  'nur',
                  'schon',
                  'vielleicht',
                  'ruhig'
                  ],#maybe
    
    'certainity':
                 ['muss',
                  'musst',
                  'müssen',
                  'müsst',
                  'muß',
                  'mußt',
                  'müßen',
                  'müßt',
                  'auf jeden fall',
                  'klipp und klar',
                  'aufjedenfall',
                  'augenscheinlich',#evidently
                  'bestimmt',#certainly
                  'definitiv',
                  'deutlich',#clearly
                  'eindeutig',
                  'gewiss',
                  'klar',
                  'offensichtlich',#obviously
                  'jedenfalls',
                  'sicher',#certainly (Nuyts 2000)
                  'sicherlich',#certainly
                  'zweifelsohne',#certainly
                  'zweifellos'#certainly
                  ],
    'mental':      ['nehme an',
                    'nimmst an',
                    'nimmt an',
                    'nehmen an',
                    'nehmt an', #assume, but with more clear qualificational use 
                    'denke',
                    'denkst',
                    'denkt',
                    'denken',
                    'glaube',
                    'glaubst',
                    'glaubt',
                    'glauben',# Nuyts 2000
                    'meine',
                    'meinst',
                    'meint',
                    'meinen', # to mean, no qualificational use in english, but stronger in german, and menen, too in dutch (Nuyts 2000)
                    'vermuten',#to presume (outdated in engish mostly, (Nuyts, 2000))
                    'vermute',
                    'vermutest',
                    'vermutet',
                    'rechne',#reckon, qulaificational use, (Nuyts 2000)
                    'rechnest',
                    'rechnet',
                    'rechnen',
                    #'sage',#while this can be used epistemically, none of the question frames do so
                    #'sagst',# and do use 'to say' in non epistemic ways
                    #'sagt',
                    #'sagen',
                    'erwarte',
                    'erwartest',
                    'erwartet',
                    'erwarten'
                  ],
     }
df['present']= df["text"].apply(lambda comment: sum(1 for w in comment.split() if w in german['present']))/df['num_words']
df['future']= df["text"].apply(lambda comment: sum(1 for w in comment.split() if w in german['future']))/df['num_words']
df['uncertainity']= df["text"].apply(lambda comment: sum(1 for w in comment.split() if w in german['uncertainity']))/df['num_words']
df['certainity']= df["text"].apply(lambda comment: sum(1 for w in comment.split() if w in german['certainity']))/df['num_words']
df['mental']= df["text"].apply(lambda comment: sum(1 for w in comment.split() if w in german['mental']))/df['num_words']
verbs=pd.read_csv("../data/top-german-verbs.csv")

In [10]:
df['Infinitiv']= df["text"].progress_apply(lambda comment: sum(1 for w in comment.split() if w in list(verbs['Infinitiv'])))/df['num_words']
df['Präsens_ich']= df["text"].progress_apply(lambda comment: sum(1 for w in comment.split() if w in list(verbs['Präsens_ich'])))/df['num_words']
df['Präsens_du']= df["text"].progress_apply(lambda comment: sum(1 for w in comment.split() if w in list(verbs['Präsens_du'])))/df['num_words']
df['Präsens_er, sie, es']= df["text"].progress_apply(lambda comment: sum(1 for w in comment.split() if w in list(verbs['Präsens_er, sie, es'])))/df['num_words']
df['Präteritum_ich']= df["text"].progress_apply(lambda comment: sum(1 for w in comment.split() if w in list(verbs['Präteritum_ich'])))/df['num_words']
df['Partizip II']= df["text"].progress_apply(lambda comment: sum(1 for w in comment.split() if w in list(verbs['Partizip II'])))/df['num_words']
df['Konjunktiv II_ich']= df["text"].progress_apply(lambda comment: sum(1 for w in comment.split() if w in list(verbs['Konjunktiv II_ich'])))/df['num_words']

  0%|          | 0/3244 [00:00<?, ?it/s]

  0%|          | 0/3244 [00:00<?, ?it/s]

  0%|          | 0/3244 [00:00<?, ?it/s]

  0%|          | 0/3244 [00:00<?, ?it/s]

  0%|          | 0/3244 [00:00<?, ?it/s]

  0%|          | 0/3244 [00:00<?, ?it/s]

  0%|          | 0/3244 [00:00<?, ?it/s]

In [11]:
df.text.fillna(" ")
df.fillna(0)

,comment_id,comment_text,Sub1_Toxic,Sub2_Engaging,Sub3_FactClaiming,user,mention,mod,total_length,num_words,...,uncertainity,certainity,mental,Infinitiv,Präsens_ich,Präsens_du,"Präsens_er, sie, es",Präteritum_ich,Partizip II,Konjunktiv II_ich
0,1,Ziemlich traurig diese Kommentare zu lesen. Ih...,0,0,0,0,0,0,322,51,...,0.078431,0.019608,0.0,0.117647,0.019608,0.0,0.039216,0.0,0.019608,0.0
1,2,"Sag ich doch, wir befeuern den Klimawandel. Ra...",0,1,1,0,0,0,148,23,...,0.173913,0.000000,0.0,0.043478,0.000000,0.0,0.000000,0.0,0.000000,0.0
2,3,"Schublade auf, Schublade zu. Zu mehr Denkleist...",1,0,0,0,0,0,79,13,...,0.076923,0.000000,0.0,0.000000,0.000000,0.0,0.076923,0.0,0.000000,0.0
3,4,Dummerweise haben wir in der EU und in der USA...,0,0,1,0,0,0,115,21,...,0.000000,0.000000,0.0,0.095238,0.000000,0.0,0.000000,0.0,0.000000,0.0
4,5,"""So lange Gewinnmaximierung Vorrang hat, wird ...",0,0,0,0,0,0,100,16,...,0.062500,0.000000,0.0,0.062500,0.125000,0.0,0.125000,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3239,3240,Hier mal eine Info. Flüchtlinge werden 10 km v...,0,0,0,0,0,0,130,22,...,0.045455,0.045455,0.0,0.090909,0.045455,0.0,0.000000,0.0,0.000000,0.0
3240,3241,@USER.aha .Mal abwarten kommt bei uns auch .Fi...,1,0,1,1,0,0,221,34,...,0.117647,0.000000,0.0,0.176471,0.029412,0.0,0.088235,0.0,0.029412,0.0
3241,3242,@USER .So ist es,0,0,0,1,0,0,16,4,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.250000,0.0,0.000000,0.0
3242,3243,@USER .Die warten da,0,0,0,1,0,0,20,4,...,0.000000,0.000000,0.0,0.250000,0.000000,0.0,0.000000,0.0,0.000000,0.0


In [12]:
import readability
def get_readability(text):
  try:
    results = readability.getmeasures(text, lang='de')
  except:
    print("error")
    return 0
  return results["readability grades"]["DaleChallIndex"]
df["readability"]=df["comment_text"].progress_apply(lambda x: get_readability(x))

  0%|          | 0/3244 [00:00<?, ?it/s]

error
error
error
error
error
error


In [13]:
df.to_csv("../data/train_ft.csv")